# Method 3: Incorporation of Adjacency Constraint with Manual Assignment of Northern Districts

## Method Details:

#### Objective Function:

- Minimize the lack of compactness (a.k.a, the sum of distances between counties that are assigned to the same district).

#### Constraints:

1. **One-County-One-District:** Each county should be assigned to exactly one district.
2. **Ideal Population:** The population in each district should be approximately equal (+- 20%) to the ideal district population size.
3. **Adjacency:** Each county in a district should be adjacency to at least one of the other counties also in that district.

#### The 3 Large Population Counties Consideration
Given that there are three counties that contain very high population around Detroit, these districts will manuaglly be assigned to their own districts.

***Approach to this consideration***
1. **Adjust the Dataset**
- Remove the three large counties, Macomb, Oakland, and Wayne from the dataset, since they will manually be assigned to their own districts.
2. **Adjust the number of districts and recalculate the average distict population**
- Since we are removing the three largest counties, we will be to recalculate the average size we want our disticts to be based on the remaining counties.
3. **Modify and Re-run the Model**
- An additional modification I made when re-running this model, what I adjusted the district population constraint to be +/- 20%. This was the only way I was able to find an optimal solution.

#### Manual Assignment of Northern Most Counties into Districts

**Manual Assignment of District 11: (Most Northern Counties)**
1. Alger
2. Baraga
3. Chippewa
4. Delta
5. Dickinson
6. Gogebic
7. Houghton
8. Iton
9. Keweenaw
10. Luce
11. Mackinac
12. Marquette
13. Menominee
14. Ontonagan
15. Schoolcraft
- Total population: 301397

**Manual Assignment of District 10: (2nd Most Northwen Counties)**
1. Emmet
2. Cheboygan
3. Presque Isle
4. Charlevoix
5. Antrim
6. Otsego
7. Montomorency
8. Alpena
- Total population: 188066


### Step 1: Load necessary libraries

In [1]:
# Import necessary libraries
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt   
import pandas as pd 
from math import pi, pow, sin, cos, asin, sqrt, floor
from pulp import *

### Step 2: Read in the Excel file and GeoJSON file
- Then merge the two data sources to have a comprehensive dataset.
    - In the 'michigan_counties' dataframe, the county name column is 'county_names', and in the 'michigan_counties_geojson' datafram, the county name column is 'name'.

In [2]:
# Read in the Excel file of Michigan counties
xlsx_file_path = '/Users/Jai/Documents/Git_remote/Decision_analytics/Module6/michigan_counties_mod.xlsx'
michigan_counties = pd.read_excel(xlsx_file_path)

# Read in the GeoJSON file of Michigan county boundaries
geojson_file_path = '/Users/Jai/Documents/Git_remote/Decision_analytics/Module6/michigan_counties_mod.geojson'
michigan_counties_geojson = gpd.read_file(geojson_file_path)

print(michigan_counties.head())
print("")
print(michigan_counties_geojson.head())

   count_id county_names   latitude  longitude  pop2020
0         0     Leelanau  45.151771 -86.038496    22870
1         1      Clinton  42.943652 -84.601517    79748
2         2      Wexford  44.338367 -85.578414    34196
3         3       Branch  41.916119 -85.059044    44531
4         4        Ionia  42.945094 -85.074603    66809

                                    geo_point_2d statefp countyfp  countyns  \
0   {'lon': -86.0384960523, 'lat': 45.151770859}      26      089  01622987   
1  {'lon': -84.6015165533, 'lat': 42.9436523662}      26      037  01622961   
2  {'lon': -85.5784138137, 'lat': 44.3383668115}      26      165  01623023   
3  {'lon': -85.0590443604, 'lat': 41.9161186535}      26      023  01622954   
4  {'lon': -85.0746031181, 'lat': 42.9450938315}      26      067  01622976   

   geoid      name         namelsad stusab lsad classfp  ... cbsafp metdivfp  \
0  26089  Leelanau  Leelanau County     MI   06      H1  ...  45900     None   
1  26037   Clinton   Clinton

In [3]:
michigan_counties['county_names'].unique()

array(['Leelanau', 'Clinton', 'Wexford', 'Branch', 'Ionia', 'Mecosta',
       'Keweenaw', 'Isabella', 'Schoolcraft', 'Crawford', 'St.Clair',
       'Missaukee', 'PresqueIsle', 'Saginaw', 'Houghton', 'VanBuren',
       'Ottawa', 'Berrien', 'Montmorency', 'Shiawassee', 'Otsego',
       'Lenawee', 'Newaygo', 'Roscommon', 'Marquette', 'Alger', 'Iron',
       'Barry', 'Emmet', 'Osceola', 'Antrim', 'Jackson', 'Manistee',
       'Calhoun', 'Tuscola', 'Gladwin', 'Menominee', 'Ontonagon',
       'Gogebic', 'Macomb', 'Midland', 'Kent', 'St.Joseph', 'Ogemaw',
       'Oceana', 'Iosco', 'Alpena', 'Sanilac', 'Oscoda', 'Washtenaw',
       'Kalamazoo', 'Ingham', 'Dickinson', 'Bay', 'Benzie', 'Huron',
       'Clare', 'Luce', 'Genesee', 'Montcalm', 'Cheboygan', 'Eaton',
       'Chippewa', 'Lake', 'Kalkaska', 'Mason', 'Mackinac', 'Oakland',
       'Monroe', 'Allegan', 'Wayne', 'Muskegon', 'Gratiot',
       'GrandTraverse', 'Baraga', 'Delta', 'Hillsdale', 'Cass', 'Lapeer',
       'Arenac', 'Charlevoix', '

In [4]:
michigan_counties_geojson['name'].unique()

array(['Leelanau', 'Clinton', 'Wexford', 'Branch', 'Ionia', 'Mecosta',
       'Keweenaw', 'Isabella', 'Schoolcraft', 'Crawford', 'St.Clair',
       'Missaukee', 'PresqueIsle', 'Saginaw', 'Houghton', 'VanBuren',
       'Ottawa', 'Berrien', 'Montmorency', 'Shiawassee', 'Otsego',
       'Lenawee', 'Newaygo', 'Roscommon', 'Marquette', 'Alger', 'Iron',
       'Barry', 'Emmet', 'Osceola', 'Antrim', 'Jackson', 'Manistee',
       'Calhoun', 'Tuscola', 'Gladwin', 'Menominee', 'Ontonagon',
       'Gogebic', 'Macomb', 'Midland', 'Kent', 'St.Joseph', 'Ogemaw',
       'Oceana', 'Iosco', 'Alpena', 'Sanilac', 'Oscoda', 'Washtenaw',
       'Kalamazoo', 'Ingham', 'Dickinson', 'Bay', 'Benzie', 'Huron',
       'Clare', 'Luce', 'Genesee', 'Montcalm', 'Cheboygan', 'Eaton',
       'Chippewa', 'Lake', 'Kalkaska', 'Mason', 'Mackinac', 'Oakland',
       'Monroe', 'Allegan', 'Wayne', 'Muskegon', 'Gratiot',
       'GrandTraverse', 'Baraga', 'Delta', 'Hillsdale', 'Cass', 'Lapeer',
       'Arenac', 'Charlevoix', '

In [5]:
# Merge the two dataframes on the county name
michigan_counties_merged = michigan_counties.merge(michigan_counties_geojson, left_on='county_names', right_on='name', how='inner')
michigan_counties_merged.head()

,count_id,county_names,latitude,longitude,pop2020,geo_point_2d,statefp,countyfp,countyns,geoid,...,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,state_name,countyfp_nozero,geometry
0,0,Leelanau,45.151771,-86.038496,22870,"{'lon': -86.0384960523, 'lat': 45.151770859}",26,089,01622987,26089,...,45900,None,A,899241895,5659105307,+45.1461816,-086.0515740,Michigan,89,"POLYGON ((-85.56175 44.95226, -85.56209 44.950..."
1,1,Clinton,42.943652,-84.601517,79748,"{'lon': -84.6015165533, 'lat': 42.9436523662}",26,037,01622961,26037,...,29620,None,A,1467017475,21098128,+42.9504550,-084.5916949,Michigan,37,"POLYGON ((-84.83762 43.03264, -84.83754 43.032..."
2,2,Wexford,44.338367,-85.578414,34196,"{'lon': -85.5784138137, 'lat': 44.3383668115}",26,165,01623023,26165,...,15620,None,A,1463148726,27182043,+44.3313751,-085.5700462,Michigan,165,"POLYGON ((-85.81909 44.42450, -85.81910 44.425..."
3,3,Branch,41.916119,-85.059044,44531,"{'lon': -85.0590443604, 'lat': 41.9161186535}",26,023,01622954,26023,...,17740,None,A,1311605515,34420092,+41.9184551,-085.0668852,Michigan,23,"POLYGON ((-85.29293 41.98482, -85.29293 41.984..."
4,4,Ionia,42.945094,-85.074603,66809,"{'lon': -85.0746031181, 'lat': 42.9450938315}",26,067,01622976,26067,...,24340,None,A,1479710906,22590318,+42.9446503,-085.0737660,Michigan,67,"POLYGON ((-85.07503 43.12021, -85.06470 43.120..."


### Step 3: Calculate the Distance Between County Pairs

In [6]:
# Define function to calculate distance between two sets of longitudes / latitudes

# Function to convert degrees to radians
def degrees_to_radians(x):
    return((pi / 180) * x)

# Function to calculate distance between two points on a sphere (in miles) given their longitudes and latitudes
def lon_lat_distance_miles(lon_a, lat_a, lon_b, lat_b):
    """
    Calculates the great-circle distance between two points on a sphere given their longitudes and latitudes.
    
    Parameters:
    lon_a (float): longitude of point A in degrees
    lat_a (float): latitude of point A in degrees
    lon_b (float): longitude of point B in degrees
    lat_b (float): latitude of point B in degrees
    
    Returns:
    float: distance between the two points in miles
    """
    radius_of_earth = 24872 / (2 * pi)
    c = sin((degrees_to_radians(lat_a) - \
    degrees_to_radians(lat_b)) / 2)**2 + \
    cos(degrees_to_radians(lat_a)) * \
    cos(degrees_to_radians(lat_b)) * \
    sin((degrees_to_radians(lon_a) - \
    degrees_to_radians(lon_b))/2)**2
    return(2 * radius_of_earth * (asin(sqrt(c))))

# Function to convert the distance between two points on a sphere (in miles) to meters
def lon_lat_distance_meters (lon_a, lat_a, lon_b, lat_b):
    return(lon_lat_distance_miles(lon_a, lat_a, lon_b, lat_b) * 1609.34)

In [7]:
# Remove population to allow easy joining of long and lat for each county pair
lat_lon = ['county_names', 'latitude', 'longitude']
lat_lon = michigan_counties_merged[lat_lon]

# Create list of county names for pairing        
county_names = michigan_counties['county_names'].to_numpy()

# Create each unique pair
pairs = []

# Loop through each county name and create a pair with each other county name
for i in range(len(county_names)):
    for j in range(i + 1, len(county_names)):
        pairs.append((county_names[i], county_names[j]))

# Create column names for county pairs df
col_names = ['county_1', 'county_2']

# Create df of county pairs                
county_pairs = pd.DataFrame(pairs, columns = col_names)

 # Add first county longitude and latitude
county_pairs = county_pairs.merge(lat_lon, left_on = 'county_1', right_on = 'county_names', how = 'left')
county_pairs.drop('county_names', axis = 1, inplace = True) # Drop county names column
county_pairs = county_pairs.rename(columns={'latitude': 'county_1_lat', 'longitude': 'county_1_long'}) # Rename columns

# Add second county longitude and latitude
county_pairs = county_pairs.merge(lat_lon, left_on = 'county_2', right_on = 'county_names', how = 'left')
county_pairs.drop('county_names', axis = 1, inplace = True) # Drop county names column
county_pairs = county_pairs.rename(columns={'latitude': 'county_2_lat', 'longitude': 'county_2_long'}) # Rename columns

# Add distance between each county pair in miles and meters;
distance_miles = [] # Create empty list to store distance in miles
distance_meters = [] # Create empty list to store distance in meters

# Loop through each county pair and calculate distance in miles and meters
for i in range(len(county_pairs)):
    distance_miles.append(lon_lat_distance_miles(county_pairs.iloc[i, 2], county_pairs.iloc[i, 3], county_pairs.iloc[i, 4], county_pairs.iloc[i, 5]))
    distance_meters.append(lon_lat_distance_meters(county_pairs.iloc[i, 2], county_pairs.iloc[i, 3], county_pairs.iloc[i, 4], county_pairs.iloc[i, 5]))

# Add distance columns to county pairs df
county_pairs['distance_miles'] = distance_miles
county_pairs['distance_meters'] = distance_meters

# Check table
county_pairs

,county_1,county_2,county_1_lat,county_1_long,county_2_lat,county_2_long,distance_miles,distance_meters
0,Leelanau,Clinton,45.151771,-86.038496,42.943652,-84.601517,100.038253,160995.562830
1,Leelanau,Wexford,45.151771,-86.038496,44.338367,-85.578414,32.050066,51579.452482
2,Leelanau,Branch,45.151771,-86.038496,41.916119,-85.059044,69.831366,112382.410744
3,Leelanau,Ionia,45.151771,-86.038496,42.945094,-85.074603,67.621439,108825.886417
4,Leelanau,Mecosta,45.151771,-86.038496,43.640768,-85.324634,49.938224,80367.581755
...,...,...,...,...,...,...,...,...
3398,Arenac,Alcona,44.042885,-83.747242,44.683623,-83.129008,43.010988,69219.303841
3399,Arenac,Livingston,44.042885,-83.747242,42.602917,-83.911528,15.593543,25095.312007
3400,Charlevoix,Alcona,45.502498,-85.373250,44.683623,-83.129008,155.151831,249692.047763
3401,Charlevoix,Livingston,45.502498,-85.373250,42.602917,-83.911528,102.674475,165238.138922


### Step 4: Create an adjacency dictionary for Michigan counties from an existing adjacency .txt file from the census website

4.1 Read in the .txt file with the adjacency information

In [8]:
# Read in the .txt file of county adjacencies
txt_file_path = '/Users/Jai/Documents/Git_remote/Decision_analytics/Module6/county_adjacency_mod.txt'

4.2 Store the Michican counties and their adjaciencies into a list

In [9]:
# Extracting the adjacencies for Michigan counties

# Parse the file to extract adjacencies for Michigan counties
adjacency_data = {}
current_county = None

with open(txt_file_path, 'r') as file:
    for line in file:
        # Split the line based on tab delimiter
        parts = line.split('\t')
        # If the line contains a county name, update the current county
        if parts[0]:
            current_county_name = parts[0].replace('"', '').strip()
            # Check if the county is from Michigan
            if "MI" in current_county_name:
                current_county = current_county_name
                adjacency_data[current_county] = []
        # If there's a current county from Michigan, append the adjacent counties to it
        elif current_county:
            adjacent_county_name = parts[2].replace('"', '').strip()
            if "MI" in adjacent_county_name:  # Only consider MI counties as adjacent
                adjacency_data[current_county].append(adjacent_county_name)

# Display the first few entries for inspection
list(adjacency_data.items())[0:3]


[('Alcona County, MI',
  ['Alpena County, MI',
   'Iosco County, MI',
   'Montmorency County, MI',
   'Ogemaw County, MI',
   'Oscoda County, MI']),
 ('Alger County, MI',
  ['Delta County, MI',
   'Keweenaw County, MI',
   'Luce County, MI',
   'Marquette County, MI',
   'Schoolcraft County, MI']),
 ('Allegan County, MI',
  ['Allegan County, MI',
   'Barry County, MI',
   'Kalamazoo County, MI',
   'Kent County, MI',
   'Ottawa County, MI',
   'VanBuren County, MI'])]

4.3 Modify this list so that only the county name e.g., "Alcona", and not the full name, e.g., "Alcona County, MI"

In [10]:
# Modifying the adjacency dictionary to retain only the county name
modified_adjacency_dict = {}

for full_name, adjacents in adjacency_data.items():
    county_name = full_name.split()[0].replace('"', '')
    modified_adjacency_dict[county_name] = [adj.split()[0].replace('"', '') for adj in adjacents]

# Checking the first few entries in the modified adjacency dictionary
list(modified_adjacency_dict.items())


[('Alcona', ['Alpena', 'Iosco', 'Montmorency', 'Ogemaw', 'Oscoda']),
 ('Alger', ['Delta', 'Keweenaw', 'Luce', 'Marquette', 'Schoolcraft']),
 ('Allegan', ['Allegan', 'Barry', 'Kalamazoo', 'Kent', 'Ottawa', 'VanBuren']),
 ('Alpena', ['Alpena', 'Montmorency', 'Oscoda', 'PresqueIsle']),
 ('Antrim',
  ['Charlevoix',
   'Crawford',
   'GrandTraverse',
   'Kalkaska',
   'Leelanau',
   'Otsego']),
 ('Arenac', ['Bay', 'Gladwin', 'Huron', 'Iosco', 'Ogemaw']),
 ('Baraga', ['Houghton', 'Iron', 'Marquette']),
 ('Barry', ['Barry', 'Calhoun', 'Eaton', 'Ionia', 'Kalamazoo', 'Kent']),
 ('Bay', ['Bay', 'Gladwin', 'Huron', 'Midland', 'Saginaw', 'Tuscola']),
 ('Benzie', ['GrandTraverse', 'Leelanau', 'Manistee', 'Wexford']),
 ('Berrien', ['Berrien', 'Cass', 'VanBuren']),
 ('Branch', ['Branch', 'Calhoun', 'Hillsdale', 'Kalamazoo', 'St.Joseph']),
 ('Calhoun',
  ['Branch',
   'Calhoun',
   'Eaton',
   'Hillsdale',
   'Jackson',
   'Kalamazoo',
   'St.Joseph']),
 ('Cass', ['Berrien', 'Cass', 'Kalamazoo', 'St.J

#### 4.4 Create the adjacency matrix for Michigan Counties

In [11]:
with open(txt_file_path, 'r') as file:
    adjacency_content = file.readlines()

# Parsing the adjacency information from the .txt file
adjacency_dict = {}
current_county = None

for line in adjacency_content:
    items = line.split("\t")
    # Check if it's a new county or an adjacent county
    if items[0]:  # New county
        current_county = items[2].split(",")[0]  # Extracting the county name
        adjacency_dict[current_county] = []
    else:  # Adjacent county
        adjacent_county = items[2].split(",")[0]  # Extracting the county name
        adjacency_dict[current_county].append(adjacent_county)

# Convert the dictionary to an adjacency matrix for Michigan counties
michigan_county_names = michigan_counties_merged['county_names'].tolist()
adjacency_matrix = pd.DataFrame(0, index=michigan_county_names, columns=michigan_county_names)

for county, neighbors in adjacency_dict.items():
    for neighbor in neighbors:
        if county in michigan_county_names and neighbor in michigan_county_names:  # Only for Michigan counties
            adjacency_matrix.at[county, neighbor] = 1

adjacency_matrix.head()

,Leelanau,Clinton,Wexford,Branch,Ionia,Mecosta,Keweenaw,Isabella,Schoolcraft,Crawford,...,GrandTraverse,Baraga,Delta,Hillsdale,Cass,Lapeer,Arenac,Charlevoix,Alcona,Livingston
Leelanau,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Clinton,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Wexford,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Branch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ionia,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Step 5: Modify the DataFrame to exclude the counties that will be manually assigned

5.1 Define the parameters for the problem

In [12]:
# Create a dictionary of county names and their populations
county_populations = michigan_counties_merged[['name', 'pop2020']].set_index('name').to_dict()['pop2020']

# Number of counties and districts in Michigan
n_counties = 83
n_districts = 14

5.2 Create lists of the large counties and the northern counites that will be manually assigned to a district

In [41]:
# Create a list of the three large counties
large_counties = ['Wayne', 'Oakland', 'Macomb']

# Create a list of the most northern counties
#most_northern = ["Alger", "Baraga", "Chippewa", "Delta", "Dickinson", "Gogebic", "Houghton", "Iron", "Keweenaw", 
         #        "Luce", "Mackinac", "Marquette", "Menominee", "Ontonagon", "Schoolcraft"]

# Create a list of the second most northern counties
#second_most_northern = ["Emmet", "Cheboygan", "PresqueIsle", "Charlevoix", "Antrim", "Otsego", 
       #                 "Montmorency", "Alpena"]

northern = ["Alger", "Baraga", "Chippewa", "Delta", "Dickinson", "Gogebic", "Houghton", "Iron", "Keweenaw","Luce", "Mackinac", "Marquette", "Menominee", "Ontonagon", "Schoolcraft", "Emmet", "Cheboygan", "PresqueIsle", "Charlevoix", "Antrim", "Otsego", 
                        "Montmorency", "Alpena" ]


# Create a list of all the manually assigned counties
manual_counties = large_counties + northern #most_northern + second_most_northern

# Create a new dataframe that doesn't contain any of the manual counties
michigan_counties_adjusted = michigan_counties_merged[~michigan_counties_merged['name'].isin(manual_counties)]



5.3 Adjust the number of counties and districts, and calculate the new ideal district population

In [49]:
# Adjust the number of counties and districts in Michigan
n_counties_adjusted = len(michigan_counties_adjusted)
print(n_counties_adjusted)
n_districts_adjusted = n_districts - (len(large_counties) + 2)

print(n_districts_adjusted)

# Approximate population of each district
average_district_population_adjusted = michigan_counties_adjusted['pop2020'].sum() / n_districts_adjusted

print(average_district_population_adjusted)

57
9
627109.0


5.4 Filter the pairs to exclude the manual counties

In [43]:
# Filter the county pairs dataframe to only include counties that are not in the large counties list
county_pairs_adjusted = county_pairs[~county_pairs['county_1'].isin(manual_counties)]

# Create a dictionary of the adjusted county names and their populations
county_populations_adjusted = michigan_counties_adjusted[['name', 'pop2020']].set_index('name').to_dict()['pop2020']
county_names_adjusted = michigan_counties_adjusted['name'].to_numpy()

5.5 Create the optimization problem

In [44]:
# Define the model
model1 = LpProblem("Michigan_Redistricting", LpMinimize)

# Define the decision variable
counties = county_names_adjusted
districts = range(n_districts_adjusted)
x = LpVariable.dicts("county_district", (counties, districts), cat='Binary')

# Objective function
# Minimize the total distance between counties in the same district
model1 += lpSum(county_pairs_adjusted.iloc[i, 7] * x[county_pairs_adjusted.iloc[i, 0]][j] for i in range(len(county_pairs_adjusted)) for j in districts)

# Constraints
# Each county must be assigned to exactly one district
for county in counties:
    model1 += lpSum(x[county][district] for district in districts) == 1

# Adjacency constraint
for county in counties:
    for district in districts:
        model1 += lpSum(adjacency_matrix.loc[county, neighbor] * x[neighbor][district] for neighbor in counties) >= x[county][district]




Added Constraint - The total population in each district should be close to the ideal district population to promote relatively equal representation

In [45]:

for j in districts:
    #For each district, ensure that the total population of the counties assigned to that district is at least 95% of the average_district_population_adjusted
    model1 += lpSum(county_populations_adjusted[county] * x[county][j] for county in counties) >= 0.95 * average_district_population_adjusted

    # ensures that the total population of the counties assigned to that district does not exceed 105% of the average_district_population_adjusted.
    model1 += lpSum(county_populations_adjusted[county] * x[county][j] for county in counties) <= 1.05 * average_district_population_adjusted

In [48]:
# Solve and print the solution
model1.solve()

# Print the status of the solution
print("Status:", LpStatus[model1.status])

# Print which counties are in each district
for district in districts:
    print("District", district + 1, ":", [county for county in counties if x[county][district].value() == 1])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/d15ed34c7b3f489c846d9222e9dce226-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/d15ed34c7b3f489c846d9222e9dce226-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 683 COLUMNS
At line 4446 RHS
At line 5125 BOUNDS
At line 5639 ENDATA
Problem MODEL has 678 rows, 513 columns and 2232 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.03

Status: Infeasible
District 1 : ['Clinton', 'St.Clair', 'Ottawa', 'Ogemaw', 'Washtenaw', 'Bay', 'Muskegon', 'GrandTraverse']
District 2 : ['M